In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-31 00:33:27.485171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Predictions (deep learning model)

In [2]:
df = pd.read_csv("embeddingsCTXencode_plus_Sbert.csv") # embeddings
df2 = pd.read_json("dataset.json") # targets

In [3]:
df2 = df2.iloc[1:] # remove first line because the embedding is title (TODO: fix csv)

In [4]:
print(df.shape, df2.shape)

(148121, 1536) (148121, 6)


In [5]:
X = np.array(df.values.tolist())
Y = pd.get_dummies(df2['category']).values  # One-hot encoding of categories

print(X.shape, Y.shape)

(148121, 1536) (148121, 15)


In [6]:
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(Y.shape[1], activation='softmax')  # Prediction head
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predicted_categories = np.argmax(predictions, axis=1)

# Display predictions
for pred, y in zip(predicted_categories[:100], np.argmax(Y_test, axis=1)):
    print(f"Target: {df2['category'].unique()[y]}, Prediction: {df2['category'].unique()[pred]}, Correctness: {pred == y}")


Epoch 1/10
3703/3703 [==============================] - 27s 7ms/step - loss: 1.0043 - accuracy: 0.6905
Epoch 2/10
3703/3703 [==============================] - 26s 7ms/step - loss: 0.8604 - accuracy: 0.7319
Epoch 3/10
3703/3703 [==============================] - 27s 7ms/step - loss: 0.8153 - accuracy: 0.7444
Epoch 4/10
3703/3703 [==============================] - 26s 7ms/step - loss: 0.7898 - accuracy: 0.7510
Epoch 5/10
3703/3703 [==============================] - 28s 8ms/step - loss: 0.7689 - accuracy: 0.7572
Epoch 6/10
3703/3703 [==============================] - 30s 8ms/step - loss: 0.7527 - accuracy: 0.7621
Epoch 7/10
3703/3703 [==============================] - 29s 8ms/step - loss: 0.7383 - accuracy: 0.7657
Epoch 8/10
3703/3703 [==============================] - 28s 8ms/step - loss: 0.7278 - accuracy: 0.7688
Epoch 9/10
3703/3703 [==============================] - 28s 8ms/step - loss: 0.7175 - accuracy: 0.7712
Epoch 10/10
926/926 [==============================] - 3s 3ms/step
Target

KeyboardInterrupt: 